# Flussbilanzanalyse mit U. maydis - Modell : Simulation und Verhältnisanalyse mit Glucose und Co-Feed von Acetat und Formiat zur CO2-negativen/neutralen Prozessgestaltung

In [1]:
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model

In [2]:
ModelFile = join('..','model','iCL1079.xml')
model=read_sbml_model(ModelFile)

model.compartments

{'e': 'Extracellular space',
 'c': 'Cytoplasm',
 'm': 'Mitochondrial Lumen',
 'p': 'Mitochondrial Intermembrane Space'}

In [ ]:
model.reactions.query('h_e')
model.add_boundary(metabolite=model.metabolites.h_e, type='exchange', reaction_id='EX_h_e', lb=0, ub=0)

 ## Reactions

### acetate

In [3]:
print('acetate reactions in cytoplasm')
for reaction in model.metabolites.ac_c.reactions:
    print(reaction.id, reaction, reaction.name)

print('')
print('acetate reactions in extracellular')
for reaction in model.metabolites.ac_e.reactions:
    print(reaction.id, reaction, reaction.name)    
# for reaction in model.metabolites.co2_c.reactions:
#     print(reaction.id, reaction, reaction.name)

# for reaction in model.metabolites.co2_e.reactions:
#     print(reaction.id, reaction, reaction.name)

acetate reactions in cytoplasm
TRANS-RXNTFS-7-ACET//ACET.11. TRANS-RXNTFS-7-ACET//ACET.11.: ac_e --> ac_c TRANS-RXNTFS-7-ACET//ACET.11.
ACKrm ACKrm: ac_c + atp_c <=> actp_c + adp_c ACETATEKIN-RXN
TRANS-RXNTFS-58-ACET/PROTON//ACET/PROTON.25. TRANS-RXNTFS-58-ACET/PROTON//ACET/PROTON.25.: ac_e + h_e --> ac_c + h_c TRANS-RXNTFS-58-ACET/PROTON//ACET/PROTON.25.
TRANS-RXNTFS-8-ACET//ACET.11. TRANS-RXNTFS-8-ACET//ACET.11.: ac_e --> ac_c TRANS-RXNTFS-8-ACET//ACET.11.
SLCYSS SLCYSS: acser_c + tsul_c <=> ac_c + h_c + slcys_c SULFOCYS-RXN
AHSERL2 AHSERL2: achms_c + h2s_c --> ac_c + h_c + hcys__L_c ACETYLHOMOSER-CYS-RXN
METB1 METB1: achms_c + cys__L_c --> ac_c + cyst__L_c + h_c RXN-721
TRANS-RXNTFS-1-ACET//ACET.11. TRANS-RXNTFS-1-ACET//ACET.11.: ac_e --> ac_c TRANS-RXNTFS-1-ACET//ACET.11.
ACSp ACSp: ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c ACETATE--COA-LIGASE-RXN
3.1.1.47-RXN 3.1.1.47-RXN: Alkyl-acetyl-glycero-ehosphocholines_c + h2o_c --> 1-Alkyl-sn-glycero-3-ehosphocholines_c + ac_c + h_c

### formate

In [4]:
print('formate reactions in cytoplasm')
for reaction in model.metabolites.for_c.reactions:
    print(reaction.id, reaction, reaction.name)
    
    
print('')
print('formate reactions in extracellular')
for reaction in model.metabolites.for_e.reactions:
    print(reaction.id, reaction, reaction.name)

formate reactions in cytoplasm
RXN-13707 RXN-13707: CPD-8606_c + 2.0 h_c + 3.0 nadph_c + 3.0 o2_c --> CPD-8609_c + for_c + 4.0 h2o_c + 3.0 nadp_c RXN-13707
1.14.13.70-RXN 1.14.13.70-RXN: OBTUSIFOLIOL_c + 2.0 h_c + 3.0 nadph_c + 3.0 o2_c --> for_c + 4.0 h2o_c + mergtrol_c + 3.0 nadp_c 1.14.13.70-RXN
GTPCII2 GTPCII2: gtp_c + 3.0 h2o_c --> 25drapp_c + for_c + 2.0 h_c + ppi_c GTP-CYCLOHYDRO-II-RXN
RXN-11453 RXN-11453: 2.0 fald_c + h2o_c <=> for_c + h_c + meoh_c RXN-11453
GTPCIn GTPCIn: gtp_c + h2o_c <=> ahdt_c + for_c + h_c GTP-CYCLOHYDRO-I-RXN
RXN-11881 RXN-11881: CPD-12852_c + 2.0 h_c + 3.0 nadph_c + 3.0 o2_c --> CPD-12853_c + for_c + 4.0 h2o_c + 3.0 nadp_c RXN-11881
RXN0-3281 RXN0-3281: for_c + nadp_c --> co2_c + nadph_c RXN0-3281
RXN3O-130 RXN3O-130: 2.0 h_c + lanost_c + 3.0 nadph_c + 3.0 o2_c --> 44-DIMETHYL-CHOLESTA-812-24-TRIENOL_c + for_c + 4.0 h2o_c + 3.0 nadp_c RXN3O-130
FORMYLTHFDEFORMYL-RXN FORMYLTHFDEFORMYL-RXN: 10fthf_c + h2o_c --> for_c + h_c + thf_c FORMYLTHFDEFORMYL-RXN
RX

In [ ]:
model.summary()
# print(model.objective)
# model.optimize()

In [ ]:
medium=model.medium
with model:
    medium['EX_glc__D_e']=7.69
#     medium['EX_ac_e']=10
    model.medium=medium
    solution=model.optimize()
    print(solution.fluxes['BIOMASS_REACTION'])
    print(model.summary())

In [ ]:
Glucose_list = np.array([5,7,9,11,13,15])
Acetate_list = np.array([0,2,4,6,8,10])
# medium_list = np.array([[x,y] for x in Glucose_list for y in Acetate_list])
Glc_list, Ace_list = np.meshgrid(Glucose_list, Acetate_list)
medium_list = np.transpose(np.reshape(np.array([Glc_list, Ace_list]),(2,-1)))
# print(medium_list)

In [ ]:
Bio_list = list()
for z in medium_list:
    x=z[0]
    y=z[1]
    with model:
        medium['EX_glc__D_e']=x
        medium['EX_ac_e']=y
        model.medium=medium
        solution=model.optimize()
        Bio_list.append(solution.fluxes['BIOMASS_REACTION'])
        
Bio_list = np.reshape(np.array(Bio_list),(-1,1))
Z = np.reshape(Bio_list,(6,6))
plt.contourf(Glc_list, Ace_list, Z, 20, cmap='coolwarm')
plt.colorbar(label='growth /h')
plt.xlabel('Glc in mmol/gCDW/h')
plt.ylabel('Ace in mmol/gCDW/h')
# plt.savefig('Umay-Glc-Ace_default.png')

## forcing proton export

In [ ]:
Bio_list = list()
for z in medium_list:
    x=z[0]
    y=z[1]
    with model:
        medium['EX_glc__D_e']=x
        medium['EX_ac_e']=y
        model.medium=medium
        model.reactions.EX_h_e.lower_bound = np.power(1.33,y)
        solution=model.optimize()
        Bio_list.append(solution.fluxes['BIOMASS_REACTION'])
        
Bio_list = np.reshape(np.array(Bio_list),(-1,1))
Z = np.reshape(Bio_list,(6,6))
plt.contourf(Glc_list, Ace_list, Z, 20, cmap='coolwarm')
plt.colorbar(label='growth /h')
plt.xlabel('Glc in mmol/gCDW/h')
plt.ylabel('Ace in mmol/gCDW/h')
plt.savefig('Umay-Glc-Ace_InhHigh.png')